In [ ]:
# default_exp exposure
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import show_doc

import pickle
import matplotlib.pyplot as plt
!date

Fri Sep  3 05:51:47 PDT 2021


# Exposure processing
> Process FT2 exposure information for a source direction 

In [ ]:
#export
import pandas as pd
import numpy as np

from wtlike.config import (Config, UTC, MJD)
from wtlike.effective_area import EffectiveArea

In [ ]:
# export
def _sc_process(config, source, sc_data):

    """
    - source -- contains ra, dec in degrees
    - sc_data -- DF constructed from spacecraft data (FT2).

    Return: a DF with the S/C data for the source direction, wtih cos theta and zenith cuts

    columns:
    - start, stop, livetime -- from the FT2 info
    - cos_theta -- angle between bore and direction
    """

    # calculate cosines with respect to sky direction
    ra_r,dec_r = np.radians(source.ra), np.radians(source.dec)
    sdec, cdec = np.sin(dec_r), np.cos(dec_r)

    def cosines( ra2, dec2):
        ra2_r =  np.radians(ra2.values)
        dec2_r = np.radians(dec2.values)
        return np.cos(dec2_r)*cdec*np.cos(ra_r-ra2_r) + np.sin(dec2_r)*sdec

    cos_thetas = cosines(sc_data.ra_scz,    sc_data.dec_scz)
    zcosines = cosines(sc_data.ra_zenith, sc_data.dec_zenith)
    # mask out entries too close to zenith, or too far away from ROI center
    mask =   (cos_thetas >= config.cos_theta_max) & (zcosines>=np.cos(np.radians(config.z_max)))
    if config.verbose>1:
        print(f'\tFound {len(mask):,} S/C entries:  {sum(mask):,} remain after zenith and theta cuts')
    dfm = sc_data.loc[mask,:]
    livetime = dfm.livetime.values

    #return livetime, cos_thetas[mask]

    return  pd.DataFrame(
        dict(
            start=sc_data.start[mask],
            stop=sc_data.stop[mask],
            livetime=livetime,
            cos_theta=cos_thetas[mask],
        )
    )

In [ ]:
# # hide
# import pickle
# import numpy as np
# from astropy.coordinates import SkyCoord
# import matplotlib.pyplot as plt
# from wtlike.data_man import get_data_files
# ff = get_data_files()
# sc_data = pickle.load(open(ff[10], 'rb'))['sc_data']

In [ ]:
# #hide
# ra, dec = 0, +45

# ra_r,dec_r = np.radians(ra), np.radians(dec)
# sdec, cdec = np.sin(dec_r), np.cos(dec_r)

# def cosines( ra2, dec2):
#     ra2_r =  np.radians(ra2.values)
#     dec2_r = np.radians(dec2.values)
#     return np.cos(dec2_r)*cdec*np.cos(ra_r-ra2_r) + np.sin(dec2_r)*sdec

# %time cos_thetas = cosines(sc_data.ra_scz,    sc_data.dec_scz)

# src = SkyCoord(ra, dec, unit='deg', frame='fk5'); src
# scdir = SkyCoord(sc_data.ra_scz.values, sc_data.dec_scz.values, unit='deg', frame='fk5')
# %time ct = np.cos(scdir.separation(src).rad)
# plt.plot(cos_thetas, ct, '.');

In [ ]:
# export
from abc import abstractmethod

class BaseExposure(object):
    """
    Base class for implementing exposure calculation
    """

    def __init__(self, config, source):

        self.config = config
        self.source = source
        self.Aeff = EffectiveArea(file_path=config.datapath/'aeff_files')
        self.setup()

    @abstractmethod
    def setup(self):
        pass

    def __call__(self, cos_theta) : #sc_data):
        """
        Apply to a SC data set
        - cos_theta -- array of cos(theta) values

        Returns:

        array of the weighted effective area

        """
        # as set by self.setup -- also serl.back_min
        edom = self.edom
        wts = self.wts #self(edom)

        # a table of the weights for each pair in livetime and cos_theta arrays
        rvals = np.empty([len(wts),len(cos_theta)])

        for i, (en,wt) in enumerate(zip(edom,wts)):
            faeff, baeff = np.array(self.Aeff( [en], cos_theta ))
            if en>self.back_min:
                rvals[i] = (faeff+baeff)*wt # note that adds front and back exposure
            else:
                rvals[i] = faeff * wt # only front

        from scipy.integrate import simpson
        aeff = simpson(rvals, edom,axis=0) / simpson(wts,edom)

        return aeff

class KerrExposure(BaseExposure):
    """
    """

    bins_per_decade: int=5
    base_spectrum: str='lambda E: (E/1000)**-2.1'
    energy_range: tuple = (100.,1e6)


    def setup(self ):

        """set up energy domain, evaluate fluxes
           This is the Kerr version, with wired-in values
        """

        emin,emax = self.energy_range
        loge1=np.log10(emin); loge2=np.log10(emax)
        self.edom= np.logspace(loge1, loge2, int((loge2-loge1)*self.bins_per_decade+1))

        spectrum = eval(self.base_spectrum) #lambda E: (E/1000)**-2.1

        self.wts = spectrum(self.edom)

        # the threshold for including Back events
        self.back_min=0

        if self.config.verbose>1:
            print(f'Set up power-law weigted exposure for {self.source.name}')

In [ ]:
# export
class SourceExposure(BaseExposure):
    """
    BaseExposure subclass that uses the source spectrum applied only to used bands
    """

    def setup(self):
        # set up weighted exposure using bands actually used, and actual flux

        wtdict = self.source.wtman.wt_dict
        bandids = np.array(list(wtdict.keys()))

        self.wts = np.array([wtdict[key]['flux'] for key in bandids if key%2==0])
        self.edom = self.config.energy_bins[:len(self.wts)]
        self.back_min = 300 # wired in--should check?

        if self.config.verbose>1:
            print(f'Set up flux-weigted exposure for {self.source.name}')

In [ ]:
# export  
def time_bin_edges(config, exposure, tbin=None):
    """Return an interleaved array of start/stop values

    tbin: an array (a,b,d), default config.time_bins

    interpretation of a, b:

        if > 50000, interpret as MJD
        if <0, back from stop
        otherwise, offset from start

    d : if positive, the day bin size
        if 0; return contiguous bins


    """
    # nominal total range, MJD edges
    start = np.round(exposure.start.values[0])
    stop =  np.round(exposure.stop.values[-1])

    a, b, step = tbin if tbin is not None else config.time_bins


    if a>50000: start=a
    elif a<0: start = stop+a
    else : start += a


    if b>50000: stop=b
    elif b>0: stop = start+b
    else: stop += b

    if step<=0:
        return contiguous_bins(exposure.query(f'{start}<start<{stop}'),)

    # adjust stop
    nbins = int((stop-start)/step)
    assert nbins>0, 'Bad binning: no bins'
    stop = start+(nbins)*step
    u =  np.linspace(start,stop, nbins+1 )

    # make an interleaved start/stop array
    v = np.empty(2*nbins, float)
    v[0::2] = u[:-1]
    v[1::2] = u[1:]
    return v

In [ ]:
# export
def sc_data_selection(config, source, sc_data):

    """
    Return a DF with the S/C data for the source direction, wtih cos theta and zenith cuts

    columns:
    - start, stop, livetime -- from the FT2 info
    - cos_theta -- angle between bore and direction
    - exp -- effective area at angle wighted by a default spectral function, times livetime

    """

    sc_df = _sc_process(config, source, sc_data)
    cos_theta = sc_df.cos_theta.values
    livetime = sc_df.livetime.values

    # now get appropriate weighted effective area, mjultipy by livetime

    if config.use_kerr:
        sc_df.loc[:,'exp'] = KerrExposure(config, source)(cos_theta) * livetime
    else:
        sc_df.loc[:,'exp']= SourceExposure(config, source)(cos_theta) * livetime

    return sc_df

In [ ]:
# export
def binned_exposure(config, exposure, time_edges):
    """Bin the exposure in to cells

    - exposure -- A DataFrame derived from FT2
    - time_bins: list of edges, an interleaved start/stop array


    Returns:

    An array of exposure integrated over each time bin. Assumes that the time bins
    are contained within the exposure.

    it is interleaved, client must apply [0::2] selection. (why not do it here?)

    """

    # get exposure calculation
    exp   =exposure.exp.values
    estart= exposure.start.values
    estop = exposure.stop.values

    # determine bins,

    #use cumulative exposure to integrate over larger periods
    cumexp = np.concatenate(([0],np.cumsum(exp)) )

    # get index into tstop array of the bin edges
    edge_index = np.searchsorted(estop, time_edges)

    # return the exposure integrated over the intervals
    cum = cumexp[edge_index]

    # difference is exposure per interval
    bexp = np.diff(cum)
#     if config.verbose>1:
#         print(f'exposure per bin:\n{pd.Series(bexp).describe(percentiles=[])}')
    return bexp

In [ ]:
show_doc(time_bin_edges)

show_doc(binned_exposure)

<h4 id="time_bin_edges" class="doc_header"><code>time_bin_edges</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>time_bin_edges</code>(**`config`**, **`exposure`**, **`tbin`**=*`None`*)

Return an interleaved array of start/stop values

tbin: an array (a,b,d), default config.time_bins

interpretation of a, b:

    if > 50000, interpret as MJD
    if <0, back from stop
    otherwise, offset from start

d : if positive, the day bin size
    if 0; return contiguous bins

<h4 id="binned_exposure" class="doc_header"><code>binned_exposure</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>binned_exposure</code>(**`config`**, **`exposure`**, **`time_edges`**)

Bin the exposure in to cells

- exposure -- A DataFrame derived from FT2
- time_bins: list of edges, an interleaved start/stop array


Returns:

An array of exposure integrated over each time bin. Assumes that the time bins
are contained within the exposure.

it is interleaved, client must apply [0::2] selection. (why not do it here?)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_exposure.ipynb.
Converted 03_sources.ipynb.
Converted 04_load_data.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_presentation.ipynb.
Converted 99_tutorial.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
Fri Sep  3 05:51:50 PDT 2021
